In [31]:
import pyodbc
import json
from tqdm import tqdm_notebook
import xlrd
from xlwt import Workbook, Formula

In [32]:
#Creating the connection (local)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost\SQLEXPRESS;'
                      'Database=pollution;'
                      'Trusted_Connection=yes;')

In [33]:
#This function insert into an Excel sheet the result of a request
#Input : the request and the name of the Excel sheet
#Output : /
def insert_excel(request, name_sheet):
    sheet = workbook.add_sheet(name_sheet)
    select_request = conn.cursor()
    select_request.execute(request)
    result = []
    for line in select_request:
        result.append(line)
    n = len(result[0])
    for j in range(n):
        for i in range(len(result)):
            sheet.write(i, j, result[i][j])

In [34]:
#Names of sheet give in Excel
name_sheets = ['req1', 'req2', 'req3', 'req4', 'req5', \
               'req6', 'req7', 'req8', 'req9', 'req10', \
               'req11', 'req12', 'req13', 'req14', 'req15', \
               'req16']

In [37]:
#This list contain all ours requests
list_request = ["Select TOP 20 label_word, sum(tf) \
                from belong b, word w \
                where w.id_word = b.id_word \
                and id_article <> 542 \
                group by label_word \
                order by 2 desc", \
               "Select TOP 20 label_word, sum(tf), id_article \
                from belong b, word w \
                where w.id_word = b.id_word \
                and id_article <> 542 \
                and tf between 5 and 30 \
                group by label_word, id_article \
                order by 2 desc", \
               "select TOP 20 sum(tf), label_word, id_article \
                from belong b, word w \
                where b.id_word = w.id_word \
                and id_article <> 542 \
                and b.is_country = 1 \
                group by label_word, id_article \
                order by 1 desc", \
               "select TOP 20 sum(tf), label_word, id_newspaper \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and b.id_article <> 542 \
                group by label_word, id_newspaper \
                order by 1 desc", \
               "select TOP 20 sum(tf), label_word, publication_date \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and b.id_article <> 542 \
                group by label_word, publication_date \
                order by 1 desc", \
               "select TOP 20 sum(tf), label_word, left(publication_date,4) \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and b.id_article <> 542 \
                and b.is_country = 1 \
                group by label_word, left(publication_date,4) \
                order by 1 desc", \
               "select TOP 50 count(label_word), id_article \
                from belong b, word w \
                where b.id_word = w.id_word \
                and id_article <> 542 \
                group by id_article \
                order by 1 desc", \
               "select TOP 20 label_word, id_article, tf \
                from belong b, word w \
                where b.id_word = w.id_word \
                and id_article <> 542 \
                and label_word in ('ocean', 'plastiques', 'plastique', 'mer', 'dechets', 'dechet', 'eau', 'pollution', \
                'microplastique') \
                order by 3 desc", \
               "select distinct TOP 20 label_word, left(publication_date, 7) \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                order by 2 desc " ,\
               "select TOP 10 count(distinct label_word), left(publication_date, 7) \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                group by left(publication_date, 7) \
                order by 1 desc", \
               "Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_description = 1 \
                group by label_word \
                order by 1 desc", \
               "Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_title = 1 \
                group by label_word \
                order by 1 desc", \
               "Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_title = 1 \
                and b.is_description = 1 \
                group by label_word \
                order by 1 desc"] 

In [38]:
#Insert into excel
workbook = Workbook()
for i in range(len(list_request)):
    request = list_request[i]
    name_sheet = name_sheets[i]
    insert_excel(request, name_sheet)
workbook.save("../database/all_result_request.xls")

# <center> FIN </center>